In [1]:
import requests
import datetime
from dateutil.relativedelta import relativedelta  # to add days or years
from datetime import date
import streamlit as st
import requests
import pandas as pd
import plotly.graph_objects as go
from bokeh.models.widgets import Div
import os
import requests
import json
import os
import sys
from collections import Counter
from os.path import dirname, exists, join
import gensim.downloader as api
import meilisearch
import nltk
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer

In [10]:
key = os.getenv('MEILI_MASTER_KEY')
headers = {'X-Meili-API-Key': "YjI1YzZhMmE4YTA0NmRjNTA5YTUxOTFi"}

In [34]:
query = ''
params = dict(q=query)

today = datetime.datetime.now()
limit_date = today + relativedelta(days=-7)
today_time = today.timestamp()
limit_time = limit_date.timestamp()



tweet_params = dict(q=query,
                    filters=f"timestamp < {limit_time}",
                    limit=100000)
tweet_params_without_query = dict(q="",
                                  filters=f"timestamp < {limit_time}")





def get_consultations():
    lexai_url = "http://35.225.139.215/indexes/consultations/search"
    result = requests.get(lexai_url, params=params, headers=headers).json()
    consultations = []
    for i in result["hits"]:
        title = i['title']
        topics = i['topics']
        type_of_act = i['type_of_act']
        try:
            status = i["status"]
        except:
            status= ""
        try:
            end_date = pd.to_datetime(i['end_date'],errors='coerce').date()
        except:
            end_date = pd.to_datetime(i['end_date'], errors='coerce')
        link = i['link']
        consultations.append({
            "title": title,
            "status": status,
            "topics": topics,
            "type_of_act": type_of_act,
            "end_date": end_date,
            "link": link
        })

    return consultations

In [27]:
get_consultations()

[{'title': 'EU cooperation on Health Technology Assessment',
  'status': 'CLOSED',
  'topics': 'Food safety',
  'type_of_act': 'PROP_REG',
  'end_date': datetime.date(2018, 4, 2),
  'link': 'https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/1565-EU-cooperation-on-Health-Technology-Assessment_en'},
 {'title': 'Amendment of the Regulation on the European Institute of Innovation and Technology (EIT)',
  'status': 'CLOSED',
  'topics': 'Culture and media',
  'type_of_act': 'PROP_REG',
  'end_date': datetime.date(2019, 9, 5),
  'link': 'https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/1530-Amendment-of-the-Regulation-on-the-European-Institute-of-Innovation-and-Technology-(EIT)_en'},
 {'title': 'Strategic Innovation Agenda for the period 2021-2027 of the European Institute of Innovation and Technology (EIT)',
  'status': 'CLOSED',
  'topics': 'Culture and media',
  'type_of_act': 'PROP_DEC',
  'end_date': datetime.date(2019, 9, 5),
  'link': '

In [59]:
def get_politicians():
    lexai_url = "http://35.225.139.215/indexes/twitter_politicians/search"
    result = requests.get(lexai_url, params=params, headers=headers).json()
    result
    info = []
    for i in result["hits"]:
        link = i["link"]
        if i["text_en"]:
            text = i["text_en"]
        else:
            text = i["text"]
        user = i["user"]
        date = i["date"]
        html_link = f'<blockquote data-cards="hidden" class="twitter-tweet" data-height="10%" data-width="100%"> <p lang="en" dir="ltr">{text}.<a href={link}</a></p>&mdash; {user} (@{user}) <a href={link}>{date}</a> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>'
        info.append({
            "link": link,
            "text": text,
            "user": user,
            "date": date,
            "html_link": html_link
        })
    return pd.DataFrame(info).sort_values(by="date",ascending=False).reset_index()

In [62]:
lexai_url = "http://35.225.139.215/indexes/twitter_politicians/search"
result = requests.get(lexai_url, params=params, headers=headers).json()
result

{'hits': [{'timestamp': 1627909703.0,
   'id': 1422212688587919364,
   'user': 'ViolavonCramon',
   'text': 'RT @sumlenny: BREAKING: Head of “Belarusian House” exile NGO in Kyiv Vitaly Shishov is missed after his morning jogging run (kidnapped?) -...',
   'text_en': 'RT @sumlenny: BREAKING: Head of “Belarusian House” exile NGO in Kyiv Vitaly Shishov is missed after his morning jogging run (kidnapped?) -...',
   'date': '2021/08/02 15:08:23',
   'lang': 'en',
   'iso_lang': None,
   'user_verified': True,
   'followers_count': 9861,
   'user_loc': 'Germany',
   'user_desc': 'Außenpolitik, Berichterstatterin #Kosovo #Wissenschaft #niedersachsen #osteuropa #erweiterung #sport #doping 💬Threema: 4VWH9BH6',
   'user_desc_en': 'Foreign policy, rapporteur #Kosovo #Science #lower Saxony #osteuropa #extension #sport #doping 💬Threema: 4VWH9BH6',
   'user_image': 'http://pbs.twimg.com/profile_images/378800000566833803/5ec2ed2f5acb4a46acfcee2266866315_normal.jpeg',
   'hashtags': '',
   'mentions':

In [37]:
params = dict(q=query, limit=100000)
lexai_url = "http://35.225.139.215/indexes/eurlex/search"
result = requests.get(lexai_url, params=params, headers=headers).json()
data_eurlex_df = pd.DataFrame(result["hits"])
data_eurlex_df

,timestamp,title,author,date,id,link
0,1.687903e+09,Consolidated text: Regulation (EU) No 575/2013...,Not available,2023/06/28,02013R057520230628,https://eur-lex.europa.eu/legal-content/AUTO/?...
1,1.653689e+09,Consolidated text: Directive 2011/83/EU of the...,Not available,2022/05/28,02011L008320220528,https://eur-lex.europa.eu/legal-content/AUTO/?...
2,1.653689e+09,Consolidated text: Directive 2005/29/EC of the...,Not available,2022/05/28,02005L002920220528,https://eur-lex.europa.eu/legal-content/AUTO/?...
3,1.640992e+09,Consolidated text: Commission Implementing Reg...,Not available,2022/01/01,02018R206620220101,https://eur-lex.europa.eu/legal-content/AUTO/?...
4,1.628719e+09,Consolidated text: Commission Implementing Reg...,Not available,2021/08/12,02012R092320210812,https://eur-lex.europa.eu/legal-content/AUTO/?...
...,...,...,...,...,...,...
2949,1.598220e+09,Proposal for a COUNCIL DECISION on the positio...,"European Commission, Directorate-General for T...",2020/08/24,52020PC0397,https://eur-lex.europa.eu/legal-content/AUTO/?...
2950,1.598220e+09,Proposal for a COUNCIL DECISION on the positio...,"European Commission, Directorate-General for T...",2020/08/24,52020PC0393,https://eur-lex.europa.eu/legal-content/AUTO/?...
2951,1.598220e+09,Proposal for a COUNCIL DECISION on the positio...,"European Commission, Directorate-General for T...",2020/08/24,52020PC0390,https://eur-lex.europa.eu/legal-content/AUTO/?...
2952,1.598220e+09,Proposal for a COUNCIL DECISION on the positio...,"European Commission, Directorate-General for T...",2020/08/24,52020PC0419,https://eur-lex.europa.eu/legal-content/AUTO/?...


In [41]:
lexai_url_news = "http://35.225.139.215/indexes/twitter_query/search"
news = requests.get(lexai_url_news, params=tweet_params,
                    headers=headers).json()

news

{'hits': [],
 'offset': 0,
 'limit': 100000,
 'nbHits': 0,
 'exhaustiveNbHits': False,
 'processingTimeMs': 0,
 'query': ''}

In [39]:
	@python LexAI/database.py build_ms_many indices=twitter_press,twitter_politicians


SyntaxError: invalid syntax (<ipython-input-39-e1660e751d21>, line 1)

In [42]:
def get_all_sentiments(self):
        folder = dirname(dirname(__file__))
        if not exists(folder):  # if folder doesn't exist
            print('Folder not found.')
            return
        
        files = os.listdir(folder)
        indices = [file.replace('.json', '') for file in files]
        
        if len(files) == 0:  # if no files
            print('No files found.')
            return
        
        for index, filename in zip(indices, files):
            filepath = join(folder, 'data.json', filename)
            with open(filepath, 'r') as file:
                documents = json.load(file)
            
            filepath = join(folder, 'update.json', f'{index}_sentiment.json')
            with open(filepath, 'w') as file:
                json.dump(self.get_sentiments(documents), file)

    
def get_sentiments(self, data):
    #initialize sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # make dict of scores
    sentiments = {i : {'id': doc['id'], 
                       'compound_score': sid.polarity_scores(doc['text_en'])['compound']}
                  for i, doc in data.items()}

    # add category
    for k, v in sentiments.items():
        if sentiments[k]['compound_score'] <= -0.2:
            sentiments[k]['sentiment'] = 'negative'
        elif sentiments[k]['compound_score'] > 0.2:
            sentiments[k]['sentiment'] = 'positive'
        else:
            sentiments[k]['sentiment'] = 'neutral'

    return sentiments



In [43]:
def extract_info(self, result, df):
        entry = {}
    
        entry['id'] = result['id']
        entry['user'] = result['user']['screen_name']
        text = normalize('NFKD', result['full_text'])
        entry['text'] = re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)
        entry['text_en'] = self.gtrans(entry['text'], dest='en')
        
        dt = datetime.strptime(result['created_at'],
                                "%a %b %d %H:%M:%S %z %Y")
        entry['date'] = datetime.strftime(dt, "%Y/%m/%d %H:%M:%S")
        entry['lang'] = result['lang']
        entry['iso_lang'] = result.get('metadata', {}).get('iso_language_code', None)
        
        entry['user_verified'] = result['user']['verified']
        entry['followers_count'] = result['user']['followers_count']
        entry['user_loc'] =  normalize('NFKD', result['user']['location'])
        entry['user_desc'] = normalize('NFKD', result['user']['description'])
        entry['user_desc_en'] = self.gtrans(entry['user_desc'], dest='en')
        entry['user_image'] = result['user']['profile_image_url']
        
        entry['hashtags'] = ', '.join([i['text'] 
                                        for i in result['entities']['hashtags']])
        entry['mentions'] = ', '.join([i['screen_name'] 
                                        for i in result['entities']['user_mentions']])
        entry['retweet_count'] = result['retweet_count']
        entry['favorite_count'] = result['favorite_count']
        
        entry['timestamp'] = mktime(dt.timetuple())
        entry['link'] = f"https://twitter.com/{entry['user']}/status/{entry['id']}"

        user = entry['user'].lower().strip()
        if all(col in df.columns for col in ['name', 'country', 'eu_group', 'national_group']):
            entry['name'] = df[df['twitter'].str.lower() == user]['name'].iloc[0]
            entry['country'] = df[df['twitter'].str.lower() == user]['country'].iloc[0]
            entry['eu_group'] = df[df['twitter'].str.lower() == user]['eu_group'].iloc[0]
            entry['national_group'] = df[df['twitter'].str.lower() == user]['national_group'].iloc[0]
        elif 'media' in df.columns:
            entry['name'] = df[df['twitter'].str.lower() == user]['media'].iloc[0]
            entry['country'] = df[df['twitter'].str.lower() == user]['country'].iloc[0]
        
        if entry['text_en'] is not None:
            sid = SentimentIntensityAnalyzer()
            entry['compound_score'] = sid.polarity_scores(entry['text_en'])['compound']
            if entry['compound_score'] <= -0.2:
                entry['sentiment'] = 'negative'
            elif entry['compound_score'] > 0.2:
                entry['sentiment'] = 'positive'
            else:
                entry['sentiment'] = 'neutral'
        
        return entry

In [50]:
url="http://35.225.139.215/indexes/twitter_politicians/search"
result = requests.get(url, params=params, headers=headers).json()
result

{'hits': [],
 'offset': 0,
 'limit': 100000,
 'nbHits': 0,
 'exhaustiveNbHits': False,
 'processingTimeMs': 0,
 'query': ''}

In [12]:
def get_all_sentiments():
        folder = dirname(dirname(os.getcwd()))+"/LexAI"
        if not exists(folder):  # if folder doesn't exist
            print('Folder not found.')
            return
        
        files = os.listdir(folder)
        indices = [file.replace('.json', '') for file in files]
        
        if len(files) == 0:  # if no files
            print('No files found.')
            return
        
        for index, filename in zip(indices, files):
            filepath = join(folder, 'data.json', filename)
            with open(filepath, 'r') as file:
                documents = json.load(file)
            
            filepath = join(folder, 'update.json', f'{index}_sentiment.json')
            with open(filepath, 'w') as file:
                json.dump(self.get_sentiments(documents), file)


In [13]:
get_all_sentiments()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/estefaniavidalbouzon/code/EstefaniaVB/LexAI/LexAI/data.json/setup.sh'

In [14]:
files = os.listdir(folder)
indices = [file.replace('.json', '') for file in files]

NameError: name 'folder' is not defined

In [15]:
files, indices

NameError: name 'files' is not defined

In [25]:
def get_all_sentiments():
    folder = os.getcwd()
    if not exists(folder):  # if folder doesn't exist
        print('Folder not found.')
        return

    files = os.listdir(os.getcwd()+"/data.json")
    indices = [file.replace('.json', '') for file in files]

    if len(files) == 0:  # if no files
        print('No files found.')
        return

    for index, filename in zip(indices, files):
        filepath = join(folder, 'data.json', filename)
        with open(filepath, 'r') as file:
            documents = json.load(file)

        filepath = join(folder, 'update.json', f'{index}_sentiment.json')
        with open(filepath, 'w') as file:
            json.dump(self.get_sentiments(documents), file)

    
def get_sentiments(data):
    #initialize sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # make dict of scores
    sentiments={}

    for n,i in enumerate(data):
        try :
            sentiments[n]={"id":i["id"],"compound_score":sid.polarity_scores(i['text_en'])['compound']}
        except:
            pass
            #sentiments[n]={"id":i["id"],"compound_score":"NaN"}
            #print(n, sentiments[n], i["id"])


    # add category
    for k, v in sentiments.items():
        if sentiments[k]['compound_score'] <= -0.2:
            sentiments[k]['sentiment'] = 'negative'
        elif sentiments[k]['compound_score'] > 0.2:
            sentiments[k]['sentiment'] = 'positive'
        else:
            sentiments[k]['sentiment'] = 'neutral'

    return sentiments

In [26]:
folder = dirname(os.getcwd())
print(folder)

files = os.listdir(dirname(os.getcwd())+"/data.json")
print(files)
indices = [file.replace('.json', '') for file in files]
print(indices)

for index, filename in zip(indices, files):
    print(index,filename)
    print("index")
    filepath = join(folder, 'data.json', filename)
    with open(filepath, 'r') as file:
        documents = json.load(file)
        print(len(documents))
    filepath = join(folder, 'update.json', f'{index}_sentiment.json')
    print("senti")
    with open(filepath, 'w') as file:
        json.dump(get_sentiments(data= documents), file)


/Users/estefaniavidalbouzon/code/EstefaniaVB/LexAI/LexAI
['twitter_query.json', 'eurlex.json', 'consultations.json', 'twitter_press.json', 'twitter_politicians.json']
['twitter_query', 'eurlex', 'consultations', 'twitter_press', 'twitter_politicians']
twitter_query twitter_query.json
index
17807
senti
eurlex eurlex.json
index
4962
senti
consultations consultations.json
index
920
senti
twitter_press twitter_press.json
index
6147
senti
twitter_politicians twitter_politicians.json
index
27369
senti


In [27]:
def extract_info(self, result, df):
        entry = {}
    
        entry['id'] = result['id']
        entry['user'] = result['user']['screen_name']
        text = normalize('NFKD', result['full_text'])
        entry['text'] = re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)
        entry['text_en'] = self.gtrans(entry['text'], dest='en')
        
        dt = datetime.strptime(result['created_at'],
                                "%a %b %d %H:%M:%S %z %Y")
        entry['date'] = datetime.strftime(dt, "%Y/%m/%d %H:%M:%S")
        entry['lang'] = result['lang']
        entry['iso_lang'] = result.get('metadata', {}).get('iso_language_code', None)
        
        entry['user_verified'] = result['user']['verified']
        entry['followers_count'] = result['user']['followers_count']
        entry['user_loc'] =  normalize('NFKD', result['user']['location'])
        entry['user_desc'] = normalize('NFKD', result['user']['description'])
        entry['user_desc_en'] = self.gtrans(entry['user_desc'], dest='en')
        entry['user_image'] = result['user']['profile_image_url']
        
        entry['hashtags'] = ', '.join([i['text'] 
                                        for i in result['entities']['hashtags']])
        entry['mentions'] = ', '.join([i['screen_name'] 
                                        for i in result['entities']['user_mentions']])
        entry['retweet_count'] = result['retweet_count']
        entry['favorite_count'] = result['favorite_count']
        
        entry['timestamp'] = mktime(dt.timetuple())
        entry['link'] = f"https://twitter.com/{entry['user']}/status/{entry['id']}"

        user = entry['user'].lower().strip()
        if all(col in df.columns for col in ['name', 'country', 'eu_group', 'national_group']):
            entry['name'] = df[df['twitter'].str.lower() == user]['name'].iloc[0]
            entry['country'] = df[df['twitter'].str.lower() == user]['country'].iloc[0]
            entry['eu_group'] = df[df['twitter'].str.lower() == user]['eu_group'].iloc[0]
            entry['national_group'] = df[df['twitter'].str.lower() == user]['national_group'].iloc[0]
        elif 'media' in df.columns:
            entry['name'] = df[df['twitter'].str.lower() == user]['media'].iloc[0]
            entry['country'] = df[df['twitter'].str.lower() == user]['country'].iloc[0]
        
        if entry['text_en'] is not None:
            sid = SentimentIntensityAnalyzer()
            entry['compound_score'] = sid.polarity_scores(entry['text_en'])['compound']
            if entry['compound_score'] <= -0.2:
                entry['sentiment'] = 'negative'
            elif entry['compound_score'] > 0.2:
                entry['sentiment'] = 'positive'
            else:
                entry['sentiment'] = 'neutral'
        
        return entry

In [70]:
files = os.listdir(dirname(os.getcwd())+"/data.json")
filepath = join(folder, 'data.json', filename)
with open(filepath, 'r') as file:
    twitter_query = json.load(file)

In [71]:
twitter_query

[{'id': 1400117440827342848,
  'timestamp': 1622641785.0,
  'user': 'PalmerReport',
  'text': 'Arizona’s Democratic Secretary of State Katie Hobbs is running for Governor of Arizona in 2022.\n\nFlorida’s Democratic Commissioner of Agriculture Nikki Fried is running for Governor of Florida in 2022.\n\nStrong women are taking over. You love to see it.',
  'text_en': 'Arizona’s Democratic Secretary of State Katie Hobbs is running for Governor of Arizona in 2022.\n\nFlorida’s Democratic Commissioner of Agriculture Nikki Fried is running for Governor of Florida in 2022.\n\nStrong women are taking over. You love to see it.',
  'date': '2021/06/02 15:49:45',
  'lang': 'en',
  'iso_lang': 'en',
  'user_verified': True,
  'followers_count': 383615,
  'user_loc': 'Los Angeles, CA',
  'user_desc': 'Political analysis ahead of the curve.',
  'user_desc_en': 'Political analysis ahead of the curve.',
  'user_image': 'http://pbs.twimg.com/profile_images/1301392629230338048/wEJnZzSn_normal.jpg',
  'ha

In [75]:
def get_sentiments(data):
    #initialize sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # make dict of scores
    sentiments = {i : {'id': i['id'], 
                       'compound_score': sid.polarity_scores(i['text_en'])['compound']}
                  for e in data for i, doc in e.items()}

    # add category
    for k, v in sentiments.items():
        if sentiments[k]['compound_score'] <= -0.2:
            sentiments[k]['sentiment'] = 'negative'
        elif sentiments[k]['compound_score'] > 0.2:
            sentiments[k]['sentiment'] = 'positive'
        else:
            sentiments[k]['sentiment'] = 'neutral'

    return sentiments


In [81]:
def get_all_sentiments():
    folder = dirname(os.getcwd())
    if not exists(folder):  # if folder doesn't exist
        print('Folder not found.')
        return

    files = os.listdir(folder)
    indices = [file.replace('.json', '') for file in files]

    if len(files) == 0:  # if no files
        print('No files found.')
        return

    for index, filename in zip(indices, files):
        filepath = join(folder, 'data.json', filename)
        with open(filepath, 'r') as file:
            documents = json.load(file)

        filepath = join(folder, 'update.json', f'{index}_sentiment.json')
        with open(filepath, 'w') as file:
            json.dump(get_sentiments(documents), file)

def get_sentiments(data):
    #initialize sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # make dict of scores
    sentiments = {i : {'id': doc['id'], 
                       'compound_score': sid.polarity_scores(doc['text_en'])} 
                  for i, doc in data.items()}

    # add category
    for k, v in sentiments.items():
        if sentiments[k]['compound_score'] <= -0.2:
            sentiments[k]['sentiment'] = 'negative'
        elif sentiments[k]['compound_score'] > 0.2:
            sentiments[k]['sentiment'] = 'positive'
        else:
            sentiments[k]['sentiment'] = 'neutral'

    return sentiments

In [82]:
get_all_sentiments()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/estefaniavidalbouzon/code/EstefaniaVB/LexAI/LexAI/data.json/setup.sh'

In [184]:
def get_all_sentiments():
        folder = dirname(dirname(os.getcwd()))+"/update.json"
        if not exists(folder):  # if folder doesn't exist
            print('Folder not found.')
            return
        
        files = os.listdir(folder)
        indices = [file.replace('.json', '') for file in files]
        
        if len(files) == 0:  # if no files
            print('No files found.')
            return
        
        for index, filename in zip(indices, files):
            filepath = join(folder, 'data.json', filename)
            with open(filepath, 'r') as file:
                documents = json.load(file)
            
            filepath = join(folder, 'update.json', f'{index}_sentiment.json')
            with open(filepath, 'w') as file:
                json.dump(self.get_sentiments(documents), file)

In [23]:
folder = dirname(os.getcwd())
if not exists(folder):  # if folder doesn't exist
    print('Folder not found.')
    

files = os.listdir(folder)
indices = [file.replace('.json', '') for file in files]

if len(files) == 0:  # if no files
    print('No files found.')
    

for index, filename in zip(indices, files):
    filepath = join(folder, 'data.json', filename)
    with open(filepath, 'r') as file:
        documents = json.load(file)

    filepath = join(folder, 'update.json', f'{index}_sentiment.json')
    with open(filepath, 'w') as file:
        json.dump(self.get_sentiments(documents), file)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/estefaniavidalbouzon/code/EstefaniaVB/LexAI/LexAI/data.json/setup.sh'